In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import backend as K
from PIL import Image

In [27]:
# Paths to the model
model_path = []
model_path.append('/content/drive/MyDrive/Machine-learning-project/Xception/Final_model/young.keras')
model_path.append('/content/drive/MyDrive/Machine-learning-project/Xception/Final_model/middle.keras')
model_path.append('/content/drive/MyDrive/Machine-learning-project/Xception/Final_model/old.keras')
model_path.append('/content/drive/MyDrive/Machine-learning-project/Xception/Final_model/F.keras')
model_path.append('/content/drive/MyDrive/Machine-learning-project/Xception/Final_model/M.keras')
model_path

['/content/drive/MyDrive/Machine-learning-project/Xception/Final_model/young.keras',
 '/content/drive/MyDrive/Machine-learning-project/Xception/Final_model/middle.keras',
 '/content/drive/MyDrive/Machine-learning-project/Xception/Final_model/old.keras',
 '/content/drive/MyDrive/Machine-learning-project/Xception/Final_model/F.keras',
 '/content/drive/MyDrive/Machine-learning-project/Xception/Final_model/M.keras']

In [30]:
sick_name = []
sick_name.append('Healthy')
sick_name.append('Effusion')
sick_name.append('Infiltration')
sick_name.append('Nodule')
sick_name.append('Atelectasis')

In [29]:
kind_name=[]
kind_name.append('young')
kind_name.append('middle')
kind_name.append('old')
kind_name.append('F')
kind_name.append('M')

In [19]:
# 從/content/drive/MyDrive/Machine-learning-project/Xception/image/的五格資料夾中分別隨機抽取10張圖片
random.seed(0)
def get_random_images(image_dir, num_images):
  image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
  if len(image_files) < num_images:
    print(f"Warning: Only {len(image_files)} images found in the directory. Returning all of them.")
    return random.sample(image_files, len(image_files))  # return all available images
  return random.sample(image_files, num_images)

image_directory = '/content/drive/MyDrive/Machine-learning-project/Xception/image/'
num_random_images = 10

random_images = []

for i in range(len(sick_name)):
  random_images.append(get_random_images(os.path.join(image_directory, sick_name[i]), num_random_images))

random_images

[['00000820_020.png',
  '00000646_007.png',
  '00000049_002.png',
  '00000547_007.png',
  '00000857_001.png',
  '00001027_001.png',
  '00000652_018.png',
  '00000627_001.png',
  '00000980_004.png',
  '00000756_005.png'],
 ['00000041_006.png',
  '00000116_008.png',
  '00000116_011.png',
  '00000071_007.png',
  '00000506_012.png',
  '00000848_000.png',
  '00000250_008.png',
  '00000591_014.png',
  '00000572_001.png',
  '00000127_004.png'],
 ['00000652_012.png',
  '00000376_010.png',
  '00001075_005.png',
  '00000548_001.png',
  '00000181_009.png',
  '00000506_003.png',
  '00001001_002.png',
  '00000796_009.png',
  '00000125_000.png',
  '00000873_000.png'],
 ['00000157_000.png',
  '00000640_001.png',
  '00000415_000.png',
  '00001012_000.png',
  '00000336_000.png',
  '00000250_003.png',
  '00000744_001.png',
  '00000105_004.png',
  '00000634_004.png',
  '00000459_017.png'],
 ['00000421_000.png',
  '00001088_008.png',
  '00000324_015.png',
  '00000307_000.png',
  '00000086_002.png',
  '000

In [24]:
# Preprocess the X-ray image
def preprocess_image(img_path, target_size=(299, 299)):
    img = load_img(img_path, target_size=target_size, color_mode='rgb')
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize to [0, 1]
    return img_array

input_image = []
for i in range(len(sick_name)):
  input_image_single_sick = []
  for j in range(len(random_images[i])):
    input_image_single_sick.append(preprocess_image(os.path.join(image_directory, sick_name[i], random_images[i][j])))
  input_image.append(input_image_single_sick)

input_image

[[array([[[[0.29411766, 0.29411766, 0.29411766],
           [0.3254902 , 0.3254902 , 0.3254902 ],
           [0.33333334, 0.33333334, 0.33333334],
           ...,
           [0.2627451 , 0.2627451 , 0.2627451 ],
           [0.29411766, 0.29411766, 0.29411766],
           [0.30588236, 0.30588236, 0.30588236]],
  
          [[0.36078432, 0.36078432, 0.36078432],
           [0.31764707, 0.31764707, 0.31764707],
           [0.30588236, 0.30588236, 0.30588236],
           ...,
           [0.32156864, 0.32156864, 0.32156864],
           [0.30588236, 0.30588236, 0.30588236],
           [0.32156864, 0.32156864, 0.32156864]],
  
          [[0.32941177, 0.32941177, 0.32941177],
           [0.33333334, 0.33333334, 0.33333334],
           [0.3254902 , 0.3254902 , 0.3254902 ],
           ...,
           [0.33333334, 0.33333334, 0.33333334],
           [0.32156864, 0.32156864, 0.32156864],
           [0.32941177, 0.32941177, 0.32941177]],
  
          ...,
  
          [[0.00392157, 0.00392157, 0.00

In [25]:
# Generate Grad-CAM
def generate_grad_cam(model, image, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(image)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap # Return the heatmap directly

In [31]:
# Identify the last convolutional layer
last_conv_layer = []
for i in range(len(model_path)):
  model = tf.keras.models.load_model(model_path[i])
  for layer in model.layers[::-1]:
      # Check if the layer has an output attribute (some layers like InputLayer might not)
      if hasattr(layer, 'output') and len(layer.output.shape) == 4:  # 4D tensor output indicates convolutional layer
          last_conv_layer.append(layer.name)
          break

  if last_conv_layer is None:
      raise ValueError("No convolutional layer found in the model.")
last_conv_layer

['block14_sepconv2_act',
 'block14_sepconv2_act',
 'block14_sepconv2_act',
 'block14_sepconv2_act',
 'block14_sepconv2_act']

In [32]:
# Generate heatmap
heatmap = []

for k in range(len(model_path)):
  model = tf.keras.models.load_model(model_path[k])
  heatmap_single_kind = []
  for i in range(len(sick_name)):
    heatmap_single_sick = []
    for j in range(len(input_image[i])):
      heatmap_single_sick.append(generate_grad_cam(model, input_image[i][j], last_conv_layer[k]))
    heatmap_single_kind.append(heatmap_single_sick)
  heatmap.append(heatmap_single_kind)

<ipython-input-25-7e6c3c571493>:17: RuntimeWarning: invalid value encountered in divide
  heatmap = np.maximum(heatmap, 0) / np.max(heatmap)


In [33]:
# Overlay Grad-CAM on the original image
def overlay_grad_cam(original_img_path, heatmap, alpha=0.4):
    original_img = Image.open(original_img_path).convert('RGB')
    original_img = original_img.resize((299, 299))
    heatmap = np.uint8(255 * heatmap)
    heatmap = Image.fromarray(heatmap).resize(original_img.size, Image.BILINEAR)
    heatmap = np.array(heatmap)
    heatmap = plt.cm.jet(heatmap / 255.0)[:, :, :3]  # Apply colormap

    overlay = np.array(original_img) / 255.0
    overlay = (1 - alpha) * overlay + alpha * heatmap
    return np.uint8(255 * overlay)

In [34]:
# overlay heatmap and image
overlay_image = []

for k in range(len(kind_name)):
  overlay_image_single_kind = []
  for i in range(len(sick_name)):
    overlay_image_single_sick = []
    for j in range(len(random_images[i])):
      overlay_image_single_sick.append(overlay_grad_cam(os.path.join(image_directory, sick_name[i], random_images[i][j]), heatmap[k][i][j]))
    overlay_image_single_kind.append(overlay_image_single_sick)
  overlay_image.append(overlay_image_single_kind)

<ipython-input-33-723cc662a091>:5: RuntimeWarning: invalid value encountered in cast
  heatmap = np.uint8(255 * heatmap)


In [35]:
output_dir_path = "/content/drive/MyDrive/Machine-learning-project/Xception/image/grad_cam_output"

for k in range(len(kind_name)):
  for i in range(len(sick_name)):
    for j in range(len(random_images[i])):
      output_path = os.path.join(output_dir_path, kind_name[k], sick_name[i], random_images[i][j])
      os.makedirs(os.path.dirname(output_path), exist_ok=True)
      Image.fromarray(overlay_image[k][i][j]).save(output_path)



# Save and display the overlayed Grad-CAM result
"""
output_path = "grad_cam_overlay.png"
Image.fromarray(overlay_image).save(output_path)
plt.figure(figsize=(10, 10))
plt.imshow(overlay_image)
plt.axis('off')
plt.title("Grad-CAM Overlay")
plt.show()
print(f"Grad-CAM overlay image saved to {output_path}.")
"""
for i in range(2):
  for j in range(2):
    for k in range(2):
      plt.figure(figsize=(10, 10))
      plt.imshow(overlay_image[i][j][k])
      plt.axis('off')
      plt.title("Grad-CAM Overlay")
      plt.show()

Output hidden; open in https://colab.research.google.com to view.